# Assignment 6

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [2]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [3]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    expected_return = np.sum(data["Probability"] * data["Price in 1 Year"])
    return (expected_return - P) / P
print(exp_return(data, 40))

0.075


Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [4]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    expected_return = np.sum(data["Probability"] * data["Price in 1 Year"])
    variance = np.sum(data["Probability"] * (data["Price in 1 Year"] - expected_return) ** 2)
    volatility = np.sqrt(variance)
    return volatility / P
print(volatility(data,40))

0.21301408404140793


Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [5]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    z_score = 1.96  # Z-score for a 95% confidence interval
    expected_return = exp_return(data, P)
    vola = volatility(data, P)
    margin_of_error = z_score * (vola / np.sqrt(len(data)))
    lower_bound = expected_return - margin_of_error
    upper_bound = expected_return + margin_of_error
    return [lower_bound, upper_bound]

print(interval(data, 40))

[-0.16604812797447732, 0.31604812797447734]


Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [6]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028
    expected_return_capm = rf + beta * (market_return - rf)
    return expected_return_capm
print(exp_return_capm(-0.6,0.1, 0.02))

-0.028


### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [7]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    data = pd.read_csv(path)

    # Convert 'RET' column to numeric, coerce non-numeric values to NaN
    data['RET'] = pd.to_numeric(data['RET'], errors='coerce')

    # Drop rows with NaN in the 'RET' column
    data = data.dropna(subset=['RET'])

    # Group by 'PERMNO' and filter out securities with fewer than 500 observations
    data = data.groupby('PERMNO').filter(lambda x: len(x) >= 500)

    # Select and reorder the columns
    data = data[['PERMNO', 'date', 'CUSIP', 'RET']]

    return data

# Example usage:
cleaned_data = clean_data("monthlycrsp_mod6.csv")
print(cleaned_data)

         PERMNO        date     CUSIP       RET
9027      10065  1970-01-30  00621210 -0.144375
9028      10065  1970-02-27  00621210  0.105882
9029      10065  1970-03-31  00621210  0.017857
9030      10065  1970-04-30  00621210 -0.078947
9031      10065  1970-05-29  00621210 -0.020952
...         ...         ...       ...       ...
3393558   84129  2022-08-31  98970110  0.016315
3393559   84129  2022-09-30  98970110 -0.075777
3393560   84129  2022-10-31  98970110  0.021235
3393561   84129  2022-11-30  98970110  0.005583
3393562   84129  2022-12-30  98970110 -0.051332

[403316 rows x 4 columns]


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [8]:
def avg_return(data):
    # Group by 'CUSIP' and calculate the mean return for each security
    avg_returns = data.groupby('CUSIP')['RET'].mean().reset_index()

    # Rename columns
    avg_returns.columns = ['CUSIP', 'Average Return']

    return avg_returns

def security_volatilities(data):
    # Group by 'CUSIP' and calculate the standard deviation (volatility) of returns for each security
    volatilities = data.groupby('CUSIP')['RET'].std().reset_index()

    # Rename columns
    volatilities.columns = ['CUSIP', 'Volatility']

    return volatilities

# Example usage:
# Assuming you have 'cleaned_data' DataFrame obtained from the previous function.
average_returns = avg_return(cleaned_data)
security_volatility = security_volatilities(cleaned_data)

# Example: Display the average returns and volatilities
print(average_returns)
print(security_volatility)

        CUSIP  Average Return
0    00036110        0.012326
1    00095710        0.011339
2    00105510        0.018398
3    00108M10        0.011101
4    00120410        0.012135
..        ...             ...
674  G6359F13        0.019803
675  G7665A10        0.013261
676  G8994E10        0.013812
677  H2717810        0.011593
678  N5946510        0.019437

[679 rows x 2 columns]
        CUSIP  Volatility
0    00036110    0.122475
1    00095710    0.089881
2    00105510    0.099828
3    00108M10    0.171732
4    00120410    0.051050
..        ...         ...
674  G6359F13    0.205250
675  G7665A10    0.129473
676  G8994E10    0.090320
677  H2717810    0.131243
678  N5946510    0.134147

[679 rows x 2 columns]


In [9]:
def avg_return(data):
    # Group by 'CUSIP' and calculate the mean return for each security
    avg_returns = data.groupby('CUSIP')['RET'].mean().reset_index()

    # Rename columns
    avg_returns.columns = ["CUSIP", "Average Return"]

    # Create an empty DataFrame with the desired format
    output = pd.DataFrame(data={"CUSIP": avg_returns["CUSIP"], "Average Return": avg_returns["Average Return"]})

    return output

# Example usage:
# Assuming you have 'cleaned_data' DataFrame obtained from the previous function.
average_returns = avg_return(cleaned_data)

# Display the result in the desired format
print(average_returns)

        CUSIP  Average Return
0    00036110        0.012326
1    00095710        0.011339
2    00105510        0.018398
3    00108M10        0.011101
4    00120410        0.012135
..        ...             ...
674  G6359F13        0.019803
675  G7665A10        0.013261
676  G8994E10        0.013812
677  H2717810        0.011593
678  N5946510        0.019437

[679 rows x 2 columns]


In [10]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    avg_returns = data.groupby('CUSIP')['RET'].mean().reset_index()
    avg_returns.columns = ["CUSIP", "Average Return"]
    

    output = pd.DataFrame(data = {
        "CUSIP" :avg_returns["CUSIP"],
        "Average Return" :avg_returns["Average Return"]
    })
    
    return output
average_returns = avg_return(cleaned_data)

# Display the result in the desired format
print(average_returns)

        CUSIP  Average Return
0    00036110        0.012326
1    00095710        0.011339
2    00105510        0.018398
3    00108M10        0.011101
4    00120410        0.012135
..        ...             ...
674  G6359F13        0.019803
675  G7665A10        0.013261
676  G8994E10        0.013812
677  H2717810        0.011593
678  N5946510        0.019437

[679 rows x 2 columns]


In [11]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    volatilities = data.groupby('CUSIP')['RET'].std().reset_index()
    volatilities.columns = ['CUSIP', 'Volatility']
    output = pd.DataFrame(data = {
        "CUSIP" : volatilities['CUSIP'],
        "Volatility" : volatilities['Volatility']
    })
    
    return output
security_volatility = security_volatilities(cleaned_data)

# Example: Display the average returns and volatilities

print(security_volatility)

        CUSIP  Volatility
0    00036110    0.122475
1    00095710    0.089881
2    00105510    0.099828
3    00108M10    0.171732
4    00120410    0.051050
..        ...         ...
674  G6359F13    0.205250
675  G7665A10    0.129473
676  G8994E10    0.090320
677  H2717810    0.131243
678  N5946510    0.134147

[679 rows x 2 columns]


Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [12]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614
    merged_data = ret_data.merge(vol_data, on='CUSIP', how='inner')


    corr = merged_data['Average Return'].corr(merged_data['Volatility'])

    return corr


corr = correlation(average_returns, security_volatility)


print(corr)    
    

0.4483709072705959
